# [Projet InPoDA - IN304](https://github.com/Egeyae/InPoDA-project-in304) - UVSQ UFR DES SCIENCES
#### *Done by KONSTANTINOV Julien and COSSEC Elouan*
---
**Goal:** Make a tweet analysis application *(extracting from french tweets: author, hastags, user mentioned, sentiment, topics)* and performing various data analysis actions

**Table of Contents**:

    - Part I: How we extract the tweets from the provided file
    - Part II: Different analysis operations performed on the tweets
    - Part III: Some references used for the project

### Installation
---
For installation process, please follow the guide in the `README.md` found in the project directory.
It is recommended to use a virtual environnement (with `Python 3.12.x` interpreter (or latest supported version by PyTorch))


In [1]:
### Setup
from InPoDA_Pipeline import *

# The InPoDA_Pipeline class is used as an interface to use the project
# A logger is set up automatically, to remove any logging/log to a file, please update the config.json file
pipeline = InPoDAPipeline()
pipeline.logger.info("Pipeline setup was a success")

/home/administrateur/kDrive/L2/InPoDA-project-in304_testinstall/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
GPU available: False
[2024-12-09 17:27:50,701] ::InPoDAPipeline:: (INFO) - Pipeline setup was a success


## I - Tweets data extraction
---
This part is a detailed explanation on how we extract tweets and parse them into a pandas.Dataframe()

##### ***1.** Load the tweets in memory*

In [2]:
tweets = pipeline.load_tweets()

[2024-12-09 17:27:50,705] ::InPoDAPipeline:: (INFO) - Loading tweets...


In [3]:
pretty_dict_display(tweets)

##### ***2.** Process the tweets in a pandas.DataFrame*

In [4]:
# Perform data extraction on the loaded tweets
dataframe = pipeline.process_tweets_to_dataframe()

[2024-12-09 17:27:50,722] ::InPoDAPipeline:: (INFO) - Processing tweets into a DataFrame...
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Tweet n°0 analysé
Tweet n°1 analysé
Tweet n°2 analysé
Tweet n°3 analysé
Tweet n°4 analysé
Tweet n°5 analysé
Tweet n°6 analysé
Tweet n°7 analysé
Tweet n°8 analysé
Tweet n°9 analysé
Tweet n°10 analysé
Tweet n°11 analysé
Tweet n°12 analysé
Tweet n°13 analysé
Tweet n°14 analysé
Tweet n°15 analysé
Tweet n°16 analysé
Tweet n°17 analysé
Tweet n°18 analysé
Tweet n°19 analysé


In [5]:
dataframe.head()

,Auteur,Hashtags,Mentions,Contenu,Topics
0,1339914264522461187,['#twitter225'],[],Goumin des éléphants joueurs la même fatigue m...,Renseignement
1,1339914264522461187,"['#SupportriceMazo', '#domie', '#CIV']","['@ericbailly24', '@maxigr04del']",@ericbailly24 @maxigr04del mes tontons vous a...,Renseignement
2,1339914264522461187,['#CIV'],[],Ah oui le sommeil là sera compliqué. #CIV est...,Renseignement
3,1339914264522461187,['#jifa'],[],"31 juillet , journée internationale de la femm...",Science des données
4,717025418,[],[],Le pedigree 😂😂😂😂 https://t.co/D3Rv7A2BOF,Arts visuels


##### *(**3.** Annex: Sentiment Analysis)*

For learning purposes, we tried to create our own Neural Network model, trained to find the sentiment of a tweet. We used a Genetic Algorithm approach to explore solutions as we were not at ease with backpropagation. The training dataset is Sentiment140, around 1.6 millions tweets annoted for sentiment analysis. We embedded the training tweets using a multilingual model as the project tweets were in French and Sentiment140's are in english.

In the following cells, we try to present the global pipeline of model usage and training. However, training the model can cost a lot in terms of resources, so the code is commented by default.

PS: As the results were too bad for any practical usage, InPoDA uses textblob for the moment until we find a valid solution. The predictions are very off the expected results, we get all fed data to be more or less in the same category. Which is strange because during training everything seems fine... An error lies between training and model usage: during training we have almost perfect results but when testing the best model we only get 50% accuracy.


###### **a.** Model loading

In [6]:
# Loads the best pre-computed model
pipeline.load_creature()

[2024-12-09 17:29:45,707] ::InPoDAPipeline:: (INFO) - Loading a pre-trained creature...


###### **b.** Model usage

In [7]:
# Example usage of the pre-computed model
test_tweet = "I'm so happy"

pipeline.process_input(test_tweet)

[2024-12-09 17:29:45,712] ::InPoDAPipeline:: (INFO) - Processing input data...


4.0


'positive'

###### (**c.** Model training)

In [ ]:
# Generates the chunks 

In [8]:
# Loads as a pandas.DataFrame the training data
# Configuration can be updated in the `config.json` file

pipeline.load_training_data()

[2024-12-09 17:29:45,819] ::InPoDAPipeline:: (INFO) - Loading training data...


FileNotFoundError: No chunks found in ./data/chunks/.

In [ ]:
# The data is split in 2: first half is negative (sentiment 0)
pipeline.data.head()

In [ ]:
# Second half is positive (sentiment 4)
pipeline.data.tail()

In [ ]:
# Here we train a model based on the loaded data
# Since it is very expensive, it doesn't run by default
# Moreover, it is preferable to run directly the script `run_training.py` found in the ./sentiment_analysis/ folder
# ! Be aware that you need to update the save file in `config.json` if you don't want to overide pre-trained model !
run = False

if run:
    pipeline.train_genetic_algorithm()

## II - Tweets data analysis
---
This part is a presentation of different analysis we can do using the data we loaded

#### **0. Data Presentation**
---
Presentation of all unique Authors, Mentions, Hashtags

In [ ]:
# All authors
pipeline.get_all_authors()

In [ ]:
# All mentions
pipeline.get_all_mentions()

In [ ]:
# All hashtags
pipeline.get_all_hashtags()

#### 1. **Top K analysis**
---
We extract:

    - Top K hastags   (Most used hashtags)
    - Top K authors   (Users who posted the most tweets)
    - Top K mentioned (Users who were the most mentioned)
    - Top K topics    (Topics that comme back the most)


In [ ]:
# Please set the desired K value here
K = 5 

In [ ]:
# TOP K HASHTAGS

In [ ]:
# TOP K AUTHORS

In [ ]:
# TOP K MENTIONED

In [ ]:
# TOP K TOPICS

#### **2. Number of tweets per X**
---
We extract:

    - Number of tweets per user
    - Number of tweets per hashtags
    - Number of tweets per topics


In [ ]:
# Number of tweets per user

In [ ]:
# Number of tweets per hashtags

In [ ]:
# Number of tweets per topics

#### **3. User analysis**
---
We extract all tweets from a provided user

In [ ]:
# Set user
user = ""

In [ ]:
# All tweets from user

#### 4. Usage analysis
---

We extract:

    - All tweets mentionning a specific user
    - All users using a specific hashtag
    - All users mentionned by a specific user

In [ ]:
# All tweets mentionning a specific user
user = ""

In [ ]:
# All users using a hashtag
hashtag = ""

In [ ]:
# All users mentionned by a specific user
user = ""

## III - References
---
Dataset:
`Sentiment140 dataset with 1.6 million tweets. (2017, September 13). https://www.kaggle.com/datasets/kazanova/sentiment140`

Genetic Algorithm:
`9. Evolutionary computing. (n.d.). https://natureofcode.com/genetic-algorithms/`